In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import sklearn.metrics as metrics
import math 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mmf-data-science-2022/sample_submission.csv
/kaggle/input/mmf-data-science-2022/train.csv
/kaggle/input/mmf-data-science-2022/test.csv


In [2]:
sample_submission = pd.read_csv("../input/mmf-data-science-2022/sample_submission.csv")
test = pd.read_csv("../input/mmf-data-science-2022/test.csv")
train = pd.read_csv("../input/mmf-data-science-2022/train.csv")
c_train = train.copy()
c_test = test.copy()                   

In [3]:
c_train.describe()
c_train.rename({'Unnamed: 0': 'id'}, axis = 1, inplace = True)
c_train.head()

,id,carat,cut,color,clarity,depth,table,price,x,y,z
0,0,1.01,Very Good,E,SI2,60.0,60.0,4540,6.57,6.49,3.92
1,1,1.10,Premium,H,VS2,62.5,58.0,5729,6.59,6.54,4.10
2,2,1.50,Good,E,SI2,61.5,65.0,6300,7.21,7.17,4.42
3,3,1.53,Premium,E,SI1,61.3,59.0,12968,7.40,7.35,4.52
4,4,0.84,Fair,D,SI2,64.5,60.0,2167,5.92,5.84,3.79


In [4]:
c_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.24,Ideal,G,VVS1,62.1,56.0,3.97,4.00,2.47
1,1,1.21,Very Good,F,VS2,62.9,54.0,6.78,6.82,4.28
2,2,0.50,Fair,E,SI1,61.7,68.0,5.09,5.03,3.12
3,3,0.50,Ideal,D,SI2,62.8,56.0,5.06,5.03,3.17
4,4,1.55,Ideal,E,SI2,62.3,55.0,7.44,7.37,4.61


In [5]:
c_train['train'] = 1
c_test['train'] = 0
df = pd.concat([c_train, c_test], axis = 0, sort = False)
df

,id,carat,cut,color,clarity,depth,table,price,x,y,z,train
0,0,1.01,Very Good,E,SI2,60.0,60.0,4540.0,6.57,6.49,3.92,1
1,1,1.10,Premium,H,VS2,62.5,58.0,5729.0,6.59,6.54,4.10,1
2,2,1.50,Good,E,SI2,61.5,65.0,6300.0,7.21,7.17,4.42,1
3,3,1.53,Premium,E,SI1,61.3,59.0,12968.0,7.40,7.35,4.52,1
4,4,0.84,Fair,D,SI2,64.5,60.0,2167.0,5.92,5.84,3.79,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10784,10784,0.53,Ideal,F,VS2,62.3,55.0,NaN,5.20,5.17,3.23,0
10785,10785,0.41,Premium,G,SI1,61.3,60.0,NaN,4.75,4.81,2.93,0
10786,10786,0.51,Premium,G,VS2,62.4,58.0,NaN,5.11,5.12,3.19,0
10787,10787,1.08,Good,J,SI2,63.2,59.0,NaN,6.40,6.57,4.10,0


In [6]:
#defining ordinal variables
cut_dict = {'Fair': 0,
           'Good': 1,
           'Very Good': 2,
           'Premium' : 3,
           'Ideal': 4}

df['cut_ordinal'] = df['cut'].map(cut_dict) 

color_dict = {'J': 0,
              'I': 1,
              'H': 2,
              'G': 3,
             'F': 4,
             'E': 5,
             'D': 6}

df['color_ordinal'] = df['color'].map(color_dict)

clarity_dict = {'I1': 0,
               'SI2': 1,
               'SI1': 2,
               'VS2': 3,
               'VS1': 4,
               'VVS2': 5,
               'VVS1': 6,
               'IF' : 7}

df['clarity_ordinal'] = df['clarity'].map(clarity_dict)

df['volume'] = df['x'] * df['y'] * df['z']

df = df.drop(['cut', 'color', 'clarity', 'x', 'y', 'z'], axis = 1)
df.head()



,id,carat,depth,table,price,train,cut_ordinal,color_ordinal,clarity_ordinal,volume
0,0,1.01,60.0,60.0,4540.0,1,2,5,1,167.146056
1,1,1.10,62.5,58.0,5729.0,1,3,2,3,176.704260
2,2,1.50,61.5,65.0,6300.0,1,1,5,1,228.494994
3,3,1.53,61.3,59.0,12968.0,1,3,5,2,245.842800
4,4,0.84,64.5,60.0,2167.0,1,0,6,1,131.030912


In [7]:
df_final = df.drop(['id'], axis =1)

df_train = df_final[df_final['train'] == 1]
df_train = df_train.drop(['train', ], axis = 1)

df_test = df_final[df_final['train'] == 0]
df_test = df_test.drop(['train',], axis  = 1 )
df_test = df_test.drop(['price', ], axis = 1)
df_test.head()

,carat,depth,table,cut_ordinal,color_ordinal,clarity_ordinal,volume
0,0.24,62.1,56.0,4,3,6,39.223600
1,1.21,62.9,54.0,2,4,3,197.905488
2,0.50,61.7,68.0,0,5,2,79.880424
3,0.50,62.8,56.0,4,6,1,80.682206
4,1.55,62.3,55.0,4,5,1,252.779208


In [8]:
df_train.head()

,carat,depth,table,price,cut_ordinal,color_ordinal,clarity_ordinal,volume
0,1.01,60.0,60.0,4540.0,2,5,1,167.146056
1,1.10,62.5,58.0,5729.0,3,2,3,176.704260
2,1.50,61.5,65.0,6300.0,1,5,1,228.494994
3,1.53,61.3,59.0,12968.0,3,5,2,245.842800
4,0.84,64.5,60.0,2167.0,0,6,1,131.030912


In [9]:
target = df_train['price']
df_train = df_train.drop(['price'], axis = 1)
df_train.head()

,carat,depth,table,cut_ordinal,color_ordinal,clarity_ordinal,volume
0,1.01,60.0,60.0,2,5,1,167.146056
1,1.10,62.5,58.0,3,2,3,176.704260
2,1.50,61.5,65.0,1,5,1,228.494994
3,1.53,61.3,59.0,3,5,2,245.842800
4,0.84,64.5,60.0,0,6,1,131.030912


In [10]:
x_train, x_test, y_train, y_test = train_test_split(df_train, target, test_size = 0.2, random_state = 0)

In [11]:
lgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=12000, 
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.4, 
                                       )

In [12]:
lgbm.fit(x_train, y_train, eval_metric = 'rmse')

[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75


LGBMRegressor(bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.4, learning_rate=0.01, max_bin=200,
              n_estimators=12000, num_leaves=4, objective='regression')

In [13]:
predict = lgbm.predict(x_test)


In [14]:
print('Root Mean Square Error test = ' + str(math.sqrt(metrics.mean_squared_error(y_test, predict))))

Root Mean Square Error test = 585.829710926501


In [15]:
lgbm.fit(df_train, target, eval_metric = 'rmse')
predict2 = lgbm.predict(df_test)

[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75


In [16]:
submission = pd.DataFrame({
    "id": test["id"],
    "price": predict2
})

submission.to_csv("submission.csv", index = False)